<a href="https://colab.research.google.com/github/MirianCabral/PySpark/blob/main/PySpark_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importação de bibliotecas/funções

In [1]:
import os
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
from pyspark.sql.window import Window # Importando window function

Criar / Iniciar Sessão Pyspark

In [3]:
spark = (
    SparkSession.builder
    .master('local')
    .appName('PySpark_02')
    .getOrCreate()
)

Criar DF/ler arquivo

In [4]:
df = spark.read.csv('/content/wc2018-players.csv', header = True, inferSchema = True)

Exibir DF

In [5]:
df.show(5)

+---------+---+----+------------------+----------+----------+--------------------+------+------+
|     Team|  #|Pos.| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
|Argentina|  3|  DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65|
|Argentina| 22|  MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65|
|Argentina| 15|  MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66|
|Argentina| 18|  DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|    69|
|Argentina| 10|  FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|   170|    72|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
only showing top 5 rows



Alterações da Aula PySpark 01

In [6]:
df = df.withColumnRenamed('Team', 'Selecao').withColumnRenamed('#', 'Numero').withColumnRenamed('Pos.', 'Posicao').withColumnRenamed('FIFA Popular Name', 'Nome_FIFA')\
.withColumnRenamed('Birth Date', 'Nascimento').withColumnRenamed('Shirt Name', 'Nome_Camiseta').withColumnRenamed('Club', 'Time')\
.withColumnRenamed('Height', 'Altura').withColumnRenamed('Weight', 'Peso')

In [7]:
dia = udf(lambda data: data.split('.')[0])
mes = udf(lambda data: data.split('.')[1])
ano = udf(lambda data: data.split('.')[2])

In [8]:
df.show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome_Camiseta|                Time|Altura|Peso|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
only showing top 5 rows



In [9]:
df = df.withColumn('Dia', dia('Nascimento')).withColumn('Mes', mes('Nascimento')).withColumn('Ano', ano('Nascimento'))
df = df.withColumn('Data_Nascimento', concat_ws('-', 'Ano', 'Mes', 'Dia').cast(DateType()))
df.show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+---+---+----+---------------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome_Camiseta|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+---+---+----+---------------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65| 31| 08|1992|     1992-08-31|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65| 21| 01|1996|     1996-01-21|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66| 15| 02|1993|     1993-02-15|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69| 13| 07|1990|     1990-07-13|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI

Drop de Colunas

In [10]:
df = df.drop('Nascimento')

In [11]:
df.show(5)

+---------+------+-------+------------------+-------------+--------------------+------+----+---+---+----+---------------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nome_Camiseta|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|
+---------+------+-------+------------------+-------------+--------------------+------+----+---+---+----+---------------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65| 31| 08|1992|     1992-08-31|
|Argentina|    22|     MF|    PAVON Cristian|        PAVÓN|CA Boca Juniors (...|   169|  65| 21| 01|1996|     1996-01-21|
|Argentina|    15|     MF|    LANZINI Manuel|      LANZINI|West Ham United F...|   167|  66| 15| 02|1993|     1993-02-15|
|Argentina|    18|     DF|    SALVIO Eduardo|       SALVIO|    SL Benfica (POR)|   167|  69| 13| 07|1990|     1990-07-13|
|Argentina|    10|     FW|      MESSI Lionel|        MESSI|  FC Barcelona (ESP)|   170|  72| 24| 06|1987|     1987-06-24|
+---------+------+------

Criar Backup

In [12]:
df2= df

###Window Ranking Functions


*   Window Function 1 - Numero de Linhas - row_number()
*   Window Function 2 - Ranking 1 - rank()
*   Window Function 3 - Ranking 2 - dense_rank()
*   Window Function 4 - Porcentagem Ranking - percent_rank()
*   Window Function 5 - Divisão em 'N' partes -ntile()



###Window Function 1 - Numero de linhas - row_number()

In [13]:
num_linha = Window.partitionBy('Selecao').orderBy(desc('Altura'))
df.withColumn('numero_linha', row_number().over(num_linha)).show(50)

+---------+------+-------+------------------+-------------+--------------------+------+----+---+---+----+---------------+------------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nome_Camiseta|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|numero_linha|
+---------+------+-------+------------------+-------------+--------------------+------+----+---+---+----+---------------+------------+
|Argentina|     6|     DF|    FAZIO Federico|        FAZIO|       AS Roma (ITA)|   199|  85| 17| 03|1987|     1987-03-17|           1|
|Argentina|     1|     GK|     GUZMAN Nahuel|       GUZMÁN|   Tigres UANL (MEX)|   192|  90| 10| 02|1986|     1986-02-10|           2|
|Argentina|    16|     DF|       ROJO Marcos|         ROJO|Manchester United...|   189|  82| 20| 03|1990|     1990-03-20|           3|
|Argentina|    12|     GK|     ARMANI Franco|       ARMANI|CA River Plate (ARG)|   189|  85| 16| 10|1986|     1986-10-16|           4|
|Argentina|    23|     GK|CABALLERO Wilfredo|    CABALL

###Window Function 2 - Ranking 1 - rank()

In [14]:
rank1 = Window.partitionBy('Selecao').orderBy(desc('Altura'))

df.withColumn('rank', rank().over(rank1)).show(50)

+---------+------+-------+------------------+-------------+--------------------+------+----+---+---+----+---------------+----+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nome_Camiseta|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|rank|
+---------+------+-------+------------------+-------------+--------------------+------+----+---+---+----+---------------+----+
|Argentina|     6|     DF|    FAZIO Federico|        FAZIO|       AS Roma (ITA)|   199|  85| 17| 03|1987|     1987-03-17|   1|
|Argentina|     1|     GK|     GUZMAN Nahuel|       GUZMÁN|   Tigres UANL (MEX)|   192|  90| 10| 02|1986|     1986-02-10|   2|
|Argentina|    16|     DF|       ROJO Marcos|         ROJO|Manchester United...|   189|  82| 20| 03|1990|     1990-03-20|   3|
|Argentina|    12|     GK|     ARMANI Franco|       ARMANI|CA River Plate (ARG)|   189|  85| 16| 10|1986|     1986-10-16|   3|
|Argentina|    23|     GK|CABALLERO Wilfredo|    CABALLERO|    Chelsea FC (ENG)|   186|  80| 28| 09|1981|     1

###Window Function 3 - Ranking 2 - dense_rank()

In [16]:
rank2 = Window.partitionBy('Selecao').orderBy(desc('Altura'))

df.withColumn('rank2', dense_rank().over(rank2)).show(50)

+---------+------+-------+------------------+-------------+--------------------+------+----+---+---+----+---------------+-----+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nome_Camiseta|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|rank2|
+---------+------+-------+------------------+-------------+--------------------+------+----+---+---+----+---------------+-----+
|Argentina|     6|     DF|    FAZIO Federico|        FAZIO|       AS Roma (ITA)|   199|  85| 17| 03|1987|     1987-03-17|    1|
|Argentina|     1|     GK|     GUZMAN Nahuel|       GUZMÁN|   Tigres UANL (MEX)|   192|  90| 10| 02|1986|     1986-02-10|    2|
|Argentina|    16|     DF|       ROJO Marcos|         ROJO|Manchester United...|   189|  82| 20| 03|1990|     1990-03-20|    3|
|Argentina|    12|     GK|     ARMANI Franco|       ARMANI|CA River Plate (ARG)|   189|  85| 16| 10|1986|     1986-10-16|    3|
|Argentina|    23|     GK|CABALLERO Wilfredo|    CABALLERO|    Chelsea FC (ENG)|   186|  80| 28| 09|1981

###Window Function 4 - Porcentagem Ranking - percent_rank()

In [18]:
porcentagem = Window.partitionBy('Selecao').orderBy(desc('Altura'))

df.withColumn('%', percent_rank().over(porcentagem)).show(50)

+---------+------+-------+------------------+-------------+--------------------+------+----+---+---+----+---------------+--------------------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nome_Camiseta|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|                   %|
+---------+------+-------+------------------+-------------+--------------------+------+----+---+---+----+---------------+--------------------+
|Argentina|     6|     DF|    FAZIO Federico|        FAZIO|       AS Roma (ITA)|   199|  85| 17| 03|1987|     1987-03-17|                 0.0|
|Argentina|     1|     GK|     GUZMAN Nahuel|       GUZMÁN|   Tigres UANL (MEX)|   192|  90| 10| 02|1986|     1986-02-10|0.045454545454545456|
|Argentina|    16|     DF|       ROJO Marcos|         ROJO|Manchester United...|   189|  82| 20| 03|1990|     1990-03-20| 0.09090909090909091|
|Argentina|    12|     GK|     ARMANI Franco|       ARMANI|CA River Plate (ARG)|   189|  85| 16| 10|1986|     1986-10-16| 0.09090909090909091|

###Window Function 5 - Divisão em 'N' partes - ntile()

In [23]:
parte = Window.partitionBy('Selecao').orderBy(desc('Altura'))

df.withColumn('partes', ntile(3).over(parte)).show(50)

+---------+------+-------+------------------+-------------+--------------------+------+----+---+---+----+---------------+------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nome_Camiseta|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|partes|
+---------+------+-------+------------------+-------------+--------------------+------+----+---+---+----+---------------+------+
|Argentina|     6|     DF|    FAZIO Federico|        FAZIO|       AS Roma (ITA)|   199|  85| 17| 03|1987|     1987-03-17|     1|
|Argentina|     1|     GK|     GUZMAN Nahuel|       GUZMÁN|   Tigres UANL (MEX)|   192|  90| 10| 02|1986|     1986-02-10|     1|
|Argentina|    16|     DF|       ROJO Marcos|         ROJO|Manchester United...|   189|  82| 20| 03|1990|     1990-03-20|     1|
|Argentina|    12|     GK|     ARMANI Franco|       ARMANI|CA River Plate (ARG)|   189|  85| 16| 10|1986|     1986-10-16|     1|
|Argentina|    23|     GK|CABALLERO Wilfredo|    CABALLERO|    Chelsea FC (ENG)|   186|  80| 28| 

###Window Analytic Functions (Funções Analíticas)

###Window Function 6 - LAG/Degrau - lag()

In [26]:
degrau = Window.partitionBy('Selecao').orderBy(desc('Altura'))
df.withColumn('degrau', lag('Altura').over(degrau)).show(50)

+---------+------+-------+------------------+-------------+--------------------+------+----+---+---+----+---------------+------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nome_Camiseta|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|degrau|
+---------+------+-------+------------------+-------------+--------------------+------+----+---+---+----+---------------+------+
|Argentina|     6|     DF|    FAZIO Federico|        FAZIO|       AS Roma (ITA)|   199|  85| 17| 03|1987|     1987-03-17|  NULL|
|Argentina|     1|     GK|     GUZMAN Nahuel|       GUZMÁN|   Tigres UANL (MEX)|   192|  90| 10| 02|1986|     1986-02-10|   199|
|Argentina|    16|     DF|       ROJO Marcos|         ROJO|Manchester United...|   189|  82| 20| 03|1990|     1990-03-20|   192|
|Argentina|    12|     GK|     ARMANI Franco|       ARMANI|CA River Plate (ARG)|   189|  85| 16| 10|1986|     1986-10-16|   189|
|Argentina|    23|     GK|CABALLERO Wilfredo|    CABALLERO|    Chelsea FC (ENG)|   186|  80| 28| 

###Window Function 7  - Lead/Degrau - lead()

In [28]:
degrau = Window.partitionBy('Selecao').orderBy(desc('Altura'))

df.withColumn('degrau', lead('Altura', 3).over(degrau)).show(50)

+---------+------+-------+------------------+-------------+--------------------+------+----+---+---+----+---------------+------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nome_Camiseta|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|degrau|
+---------+------+-------+------------------+-------------+--------------------+------+----+---+---+----+---------------+------+
|Argentina|     6|     DF|    FAZIO Federico|        FAZIO|       AS Roma (ITA)|   199|  85| 17| 03|1987|     1987-03-17|   189|
|Argentina|     1|     GK|     GUZMAN Nahuel|       GUZMÁN|   Tigres UANL (MEX)|   192|  90| 10| 02|1986|     1986-02-10|   186|
|Argentina|    16|     DF|       ROJO Marcos|         ROJO|Manchester United...|   189|  82| 20| 03|1990|     1990-03-20|   184|
|Argentina|    12|     GK|     ARMANI Franco|       ARMANI|CA River Plate (ARG)|   189|  85| 16| 10|1986|     1986-10-16|   181|
|Argentina|    23|     GK|CABALLERO Wilfredo|    CABALLERO|    Chelsea FC (ENG)|   186|  80| 28| 

###Agregações

###GroupBy = AGG

In [40]:
df.groupBy('Selecao').agg(avg('Altura')).orderBy(('avg(Altura)'), ascending=False).show(50)

+--------------+------------------+
|       Selecao|       avg(Altura)|
+--------------+------------------+
|        Serbia|186.69565217391303|
|       Denmark| 186.6086956521739|
|       Germany| 185.7826086956522|
|        Sweden| 185.7391304347826|
|       Iceland|185.52173913043478|
|       Belgium|185.34782608695653|
|       Croatia| 185.2608695652174|
|       Nigeria|184.52173913043478|
|       IR Iran|184.47826086956522|
|        Russia| 184.3913043478261|
|       Senegal|183.65217391304347|
|        France|183.30434782608697|
|        Poland|183.17391304347825|
|       Tunisia|183.08695652173913|
|   Switzerland|182.91304347826087|
|       England| 182.7391304347826|
|       Morocco|182.69565217391303|
|        Panama|182.17391304347825|
|Korea Republic| 181.8695652173913|
|       Uruguay|181.04347826086956|
|         Egypt|             181.0|
|     Australia| 180.8695652173913|
|        Brazil| 180.7826086956522|
|      Colombia| 180.7826086956522|
|    Costa Rica|180.69565217

###Where

In [41]:
df.where('Selecao = "Brazil"').show(25)

+-------+------+-------+-----------------+-------------+--------------------+------+----+---+---+----+---------------+
|Selecao|Numero|Posicao|        Nome_FIFA|Nome_Camiseta|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|
+-------+------+-------+-----------------+-------------+--------------------+------+----+---+---+----+---------------+
| Brazil|    18|     MF|             FRED|         FRED|FC Shakhtar Donet...|   169|  64| 05| 03|1993|     1993-03-05|
| Brazil|    21|     FW|           TAISON|       TAISON|FC Shakhtar Donet...|   172|  64| 13| 01|1988|     1988-01-13|
| Brazil|    17|     MF|      FERNANDINHO|  FERNANDINHO|Manchester City F...|   179|  67| 04| 05|1985|     1985-05-04|
| Brazil|    22|     DF|           FAGNER|       FAGNER|SC Corinthians (BRA)|   168|  67| 11| 06|1989|     1989-06-11|
| Brazil|    10|     FW|           NEYMAR|    NEYMAR JR|Paris Saint-Germa...|   175|  68| 05| 02|1992|     1992-02-05|
| Brazil|    11|     MF|PHILIPPE COUTINHO|  P. C

In [45]:
top1 = Window.partitionBy('Selecao').orderBy(desc('Altura'))

df.withColumn('Top', row_number().over(top1)).where('Top = "1"').show(50)

+--------------+------+-------+-------------------+-------------+--------------------+------+----+---+---+----+---------------+---+
|       Selecao|Numero|Posicao|          Nome_FIFA|Nome_Camiseta|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|Top|
+--------------+------+-------+-------------------+-------------+--------------------+------+----+---+---+----+---------------+---+
|     Argentina|     6|     DF|     FAZIO Federico|        FAZIO|       AS Roma (ITA)|   199|  85| 17| 03|1987|     1987-03-17|  1|
|     Australia|    12|     GK|         JONES Brad|        JONES|Feyenoord Rotterd...|   193|  87| 19| 03|1982|     1982-03-19|  1|
|       Belgium|     1|     GK|   COURTOIS Thibaut|     COURTOIS|    Chelsea FC (ENG)|   199|  91| 11| 05|1992|     1992-05-11|  1|
|        Brazil|    16|     GK|             CASSIO|       CASSIO|SC Corinthians (BRA)|   195|  92| 06| 06|1987|     1987-06-06|  1|
|      Colombia|    13|     DF|         MINA Yerry|      Y. MINA|  FC Barcel

###Describe

In [47]:
df.describe().show()

+-------+---------+-----------------+-------+------------+-------------+--------------------+-----------------+-----------------+------------------+------------------+------------------+
|summary|  Selecao|           Numero|Posicao|   Nome_FIFA|Nome_Camiseta|                Time|           Altura|             Peso|               Dia|               Mes|               Ano|
+-------+---------+-----------------+-------+------------+-------------+--------------------+-----------------+-----------------+------------------+------------------+------------------+
|  count|      736|              736|    736|         736|          736|                 736|              736|              736|               736|               736|               736|
|   mean|     NULL|             12.0|   NULL|        NULL|         NULL|                NULL|182.4076086956522|77.18885869565217|15.793478260869565|5.8790760869565215| 1990.110054347826|
| stddev|     NULL|6.637760461599851|   NULL|        NULL|       

In [48]:
df.where('Selecao = "Brazil"').describe().show()

+-------+-------+-----------------+-------+---------+-------------+--------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|Selecao|           Numero|Posicao|Nome_FIFA|Nome_Camiseta|                Time|           Altura|             Peso|              Dia|              Mes|              Ano|
+-------+-------+-----------------+-------+---------+-------------+--------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|     23|               23|     23|       23|           23|                  23|               23|               23|               23|               23|               23|
|   mean|   NULL|             12.0|   NULL|     NULL|         NULL|                NULL|180.7826086956522|76.56521739130434|11.26086956521739|6.130434782608695|1989.391304347826|
| stddev|   NULL|6.782329983125267|   NULL|     NULL|         NULL|                NULL|7.354383490255254

###Window Function 8 - Função de agregação usando Window Function

In [51]:
parametro = Window.partitionBy('Selecao').orderBy(desc('Altura'))
parametro2 = Window.partitionBy('Selecao')
df.withColumn('linhax', row_number().over(parametro))\
.withColumn('media', avg('Altura').over(parametro2))\
.withColumn('max', max('Altura').over(parametro2))\
.withColumn('min', min('Altura').over(parametro2))\
.where('linhax = "1"').select('Selecao', 'media', 'max', 'min')\
.orderBy('media', ascending=False).show(50)

+--------------+------------------+---+---+
|       Selecao|             media|max|min|
+--------------+------------------+---+---+
|        Serbia|186.69565217391303|195|169|
|       Denmark| 186.6086956521739|200|171|
|       Germany| 185.7826086956522|195|176|
|        Sweden| 185.7391304347826|198|177|
|       Iceland|185.52173913043478|198|170|
|       Belgium|185.34782608695653|199|169|
|       Croatia| 185.2608695652174|201|172|
|       Nigeria|184.52173913043478|197|172|
|       IR Iran|184.47826086956522|194|177|
|        Russia| 184.3913043478261|196|173|
|       Senegal|183.65217391304347|196|173|
|        France|183.30434782608697|197|168|
|        Poland|183.17391304347825|195|172|
|       Tunisia|183.08695652173913|192|170|
|   Switzerland|182.91304347826087|192|165|
|       England| 182.7391304347826|196|170|
|       Morocco|182.69565217391303|190|167|
|        Panama|182.17391304347825|197|165|
|Korea Republic| 181.8695652173913|197|170|
|       Uruguay|181.043478260869